In [1]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.3 MB/s 
     |████████████████████████████████| 1.6 MB 33.1 MB/s 


In [11]:
import math
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model 
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler 
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [4]:
TRAIN_DATA_PATH = 'california_housing_train.csv'
TEST_DATA_PATH = 'california_housing_test.csv'
TARGET_NAME = 'median_house_value'

In [7]:
train_data= pd.read_csv(TRAIN_DATA_PATH)
test_data= pd.read_csv(TEST_DATA_PATH)

In [8]:
x_train, y_train = train_data.drop(TARGET_NAME, axis=1), train_data[TARGET_NAME]
x_test, y_test = test_data.drop(TARGET_NAME, axis=1), test_data[TARGET_NAME]

In [10]:
def scale_datasets(x_train, x_test):
  standard_scaler = StandardScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )

  x_test_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_test),
      columns=x_test.columns      
  )
  return x_train_scaled, x_test_scaled 

x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [17]:
import kerastuner as kt
msle = MeanSquaredLogarithmicError()

def build_model(hp):
  model = tf.keras.Sequential()
  
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(Dense(units=hp_units1, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='linear'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
      loss=msle,
      metrics=[msle]
  )

  return model

In [18]:
tuner = kt.Hyperband(
    build_model,
    objective='val_mean_squared_logarithmic_error',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo'
)

In [19]:
tuner.search(x_train_scaled, y_train, epochs=10, validation_split=0.2)

Trial 30 Complete [00h 00m 42s]
val_mean_squared_logarithmic_error: 0.1224481537938118

Best val_mean_squared_logarithmic_error So Far: 0.10669773072004318
Total elapsed time: 00h 05m 13s


In [20]:
for h_param in [f"units{i}" for i in range(1,4)] + ['learning_rate']:
  print(h_param, tuner.get_best_hyperparameters()[0].get(h_param))

units1 288
units2 448
units3 448
learning_rate 0.01


In [21]:
best_model = tuner.get_best_models()[0]
best_model.build(x_train_scaled.shape)
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (17000, 288)              2592      
                                                                 
 dense_1 (Dense)             (17000, 448)              129472    
                                                                 
 dense_2 (Dense)             (17000, 448)              201152    
                                                                 
 dense_3 (Dense)             (17000, 1)                449       
                                                                 
Total params: 333,665
Trainable params: 333,665
Non-trainable params: 0
_________________________________________________________________


In [22]:
best_model.fit(
    x_train_scaled,
    y_train,
    epochs=10,
    batch_size=64
)

Epoch 1/10
266/266 [==============================] - 3s 10ms/step - loss: 0.0905 - mean_squared_logarithmic_error: 0.0906
Epoch 2/10
266/266 [==============================] - 2s 8ms/step - loss: 0.0894 - mean_squared_logarithmic_error: 0.0894
Epoch 3/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0883 - mean_squared_logarithmic_error: 0.0883
Epoch 4/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0879 - mean_squared_logarithmic_error: 0.0879
Epoch 5/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0865 - mean_squared_logarithmic_error: 0.0864
Epoch 6/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0853 - mean_squared_logarithmic_error: 0.0854
Epoch 7/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0844 - mean_squared_logarithmic_error: 0.0845
Epoch 8/10
266/266 [==============================] - 2s 7ms/step - loss: 0.0833 - mean_squared_logarithmic_error: 0.0833
Epoch 9/10
266/266 [===

In [23]:
msle(y_test, best_model.predict(x_test_scaled)).numpy()

0.54122543